In [4]:
# Import necessary libraries
import pandas as pd
import yfinance as yf

# Define the start and end dates for the stock data to be retrieved
start = '2019-09-01'
end = '2024-11-09'

# Prompt for Stock1 ticker input
while True:
    
    stock1_ticker = input('Enter Stock1:')
    
    # Check if input is empty
    if len(stock1_ticker) == 0:
        print('Error: Input must not be empty. Please enter a valid ticker.')
        continue
        
    # Check if input is a number (not valid for tickers)
    if stock1_ticker.isdigit():
        print('Error: Ticker symbol cannot be a number. Please enter a valid ticker')
        continue
        
    # If input is valid, break out of the loop
    break

# Same steps for Stock2
while True:
    
    stock2_ticker = input('Enter Stock2:')

    if len(stock2_ticker) == 0:
        print('Error: Input must not be empty. Please enter a valid ticker.')
        continue

    if stock2_ticker.isdigit():
        print('Error: Ticker symbol cannot be a number. Please enter a valid ticker')
        continue

    break

# Download stock data for two stocks specified by the user input. User will be prompted to enter in ticker symbols via the 'input' function.
# Assign variable names stock1 and stock2

stock1 = yf.download(stock1_ticker, start=start, end=end)
stock2 = yf.download(stock2_ticker, start=start, end=end)

# Merge the two stock dataframes on the 'Date' column, adding suffixes to distinguish between the two stocks
pair = pd.merge(stock1, stock2, on='Date', suffixes=('_stock1', '_stock2'))

# Calculate the spread between the opening prices of the two stocks
pair['Spread Open'] = pair['Open_stock1'] / pair['Open_stock2']

# Calculate the spread between the closing prices of the two stocks
pair['Spread Close'] = pair['Close_stock1'] / pair['Close_stock2']

# Calculate the percentage change in the spread from the previous day's close to today's open
pair['Spread Close to Open'] = (pair['Spread Open'] - pair['Spread Close'].shift(1)) / pair['Spread Close'].shift(1) * 100

# Calculate the percentage change in the spread from today's open to today's close
pair['Spread Open to Close'] = (pair['Spread Close'] - pair['Spread Open']) / pair['Spread Open'] * 100

# Prompt the user to input a gap percentage to filter based on the degree of the gap at the open
gappct = float(input('Gap %:'))
print()

# Filter the pairs where the percentage change in the spread exceeds the specified gap percentage
gap = pair[(pair['Spread Close to Open'] > gappct - 0.2) & (pair['Spread Close to Open'] < gappct + 0.2)]

# Separate the filtered data into positive and negative gaps
positivegap = gap[gap['Spread Close to Open'] > 0]
negativegap = gap[gap['Spread Close to Open'] < 0]

# Calculate the total performance of the pairs on positive gaps
sum_gap = gap['Spread Open to Close'].sum()
# Calculate the average performance of the pairs on positive gaps
avg_gap = gap['Spread Open to Close'].mean()
# Calculate the total performance of the pairs on negative gaps

# Count the number of occurrences where the spread reverts negatively after a positive gap
if gappct > 0:
    num_neg_outcomes_up_gap = len(positivegap[positivegap['Spread Open to Close'] < 0])
    # Calculate the odds of the pair reverting after a positive gap
    odds_gap_up = round(num_neg_outcomes_up_gap / len(positivegap) * 100, 1)
    
# Count the number of occurrences where the spread reverts positively after a negative gap
else:
    num_pos_outcomes_down_gap = len(negativegap[negativegap['Spread Open to Close'] > 0])
    # Calculate the odds of the pair reverting gap
    odds_gap_down = round(num_pos_outcomes_down_gap / len(negativegap) * 100, 1)


# Print the results for gap performance
print('Total Gap Performance:', sum_gap)
print('Average Gap Performance:', avg_gap)
if gappct > 0:
    print(f'Odds of Pair Reverting on Gap: {odds_gap_up}%\n')
    print(f'')
else:
    print(f'Odds of Pair Reverting on Negative Gap: {odds_gap_down}%')


Enter Stock1: iwm
Enter Stock2: rsp


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Gap %: -0.25



Total Gap Performance: -8.564762196286704
Average Gap Performance: -0.019689108497210814
Odds of Pair Reverting on Negative Gap: 47.4%
